In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

# Banking and Poverty
---

In [1]:
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt


# Census API Key
c = Census(census_key, year=2013)

## Data Retrieval

In [7]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B17001_002E"), {
                         'for': 'zip code tabulation area:*'})

# # Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# # Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd['Poverty Rate']=census_pd['Poverty Count']/census_pd['Population']*100
# Final DataFrame


# Visualize
census_pd.head()

,Population,Poverty Count,Zipcode,Poverty Rate
0,22121.0,2412.0,01832,10.903666
1,8295.0,191.0,01833,2.302592
2,6675.0,146.0,01834,2.187266
3,13527.0,1153.0,01835,8.523693
4,4547.0,1798.0,01840,39.542556


In [4]:
census_pd['Zipcode']

Population       float64
Poverty Count    float64
Zipcode           object
dtype: object

## Combine Data

In [8]:
# Import the zip bank data. Use dtype="object" to ensure all data is read in accurately.
bank_data_df=pd.read_csv('../Resources/zip_bank_data.csv')

# Visualize
bank_data_df.head()

,Zipcode,Address,Median Age,Household Income,Per Capita Income,Lat,Lng,Bank Count
0,27358,"Summerfield, NC 27358, USA",43.0,88382.0,40210,36.234502,-79.885682,4
1,14113,"North Java, NY 14113, USA",45.6,47500.0,33149,42.683985,-78.327325,0
2,71825,"Blevins, AR 71825, USA",32.6,35625.0,13098,33.868278,-93.550907,2
3,32535,"Century, FL 32535, USA",37.8,34107.0,13066,30.972476,-87.371148,0
4,5483,"Sheldon, VT 05483, USA",35.9,65039.0,24743,46.738660,-92.282514,2


In [9]:
# Merge the two data sets along zip code
complete_df=pd.merge(bank_data_df, census_pd, how='left', on='Zipcode')

# Remove rows missing data
complete_df=complete_df.dropna()

# Visualize
complete_df.head()

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

## Heatmap of poverty rate

In [ ]:
# Configure gmaps with API key

In [ ]:
# Store 'Lat' and 'Lng' into  locations 

# Convert Poverty Rate to float and store


In [ ]:
# Create a poverty Heatmap layer


In [ ]:
# Convert bank rate to list


In [ ]:
# Create bank symbol layer


In [ ]:
# Create a combined map


## Statistical Analysis

#### Summary Statistics

In [ ]:
# Mean, median, mode for Poverty Rate, Bank Count, Population

In [ ]:
# Mean, median, mode for Poverty Rate

In [ ]:
# Mean, median, mode for Bank Count

In [ ]:
# Mean, median, mode for Population

#### Linear Regression

In [ ]:
## Convert to floats and store Poverty Rate and Bank Count as x and y values


# Run linear regression


# Plot scatter plot


# Plot regression line


# Label plot


# Print r square value


# Show plot


### Analysis